In [1]:
pip install ipynb --upgrade

Requirement already up-to-date: ipynb in /Users/mattiaskallman1/anaconda3/lib/python3.7/site-packages (0.5.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd

os.chdir("..")
import Models
os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
import ipynb.fs.full.misc as misc
import ipynb.fs.full.splitting as splitting
import ipynb.fs.full.table as table
import ipynb.fs.full.decide as decide
import ipynb.fs.full.real_features as features
os.chdir("../models-decision-system/MockPipeline")

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.python.keras import Input, Model

from tcn import TCN, tcn_full_summary
#os.getcwd()

In [3]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [4]:
config = load_yaml('mock_settings.yaml')

In [5]:
dataframe = processing.create_dataframe(config)

In [6]:
dataset = features.add(dataframe, config['features'])

In [7]:
features = dataset.loc[:, dataset.columns != 'label'].to_numpy()
labels = dataset[['label']].to_numpy()
primary, scaler = splitting.primary(features, labels, config['splitting']['train_split'])

In [8]:
linreg_fold = splitting.timeseries(
            primary['train'],
            config['splitting']['validation_folds']
        )

In [21]:
len(linreg_fold)

5

In [9]:
settings = config['regression_ensemble']['models'][1]['lstm']
lstm_fold = splitting.timeseries(
            primary['train'],
            config['splitting']['validation_folds'],
            window=settings['morph']['window']
        )

## I John's kod så börjar ensemble loopen här

In [10]:
# WORKING
linreg = Models.train_model(linreg_fold[0], 'linreg', config['regression_ensemble']['models'][0])

In [11]:
linreg

{'model': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 'predictions': array([[1.41541475],
        [1.41107816],
        [1.41932893],
        [1.43397511],
        [1.42277239],
        [1.43098948],
        [1.44161477],
        [1.43247798],
        [1.45117508],
        [1.46561695],
        [1.46642296],
        [1.46165891],
        [1.47083355],
        [1.45986677],
        [1.45386093],
        [1.43478465],
        [1.43511121],
        [1.44410573],
        [1.44483765],
        [1.42056489],
        [1.42487372],
        [1.43260183],
        [1.42896062],
        [1.43867974],
        [1.4389763 ],
        [1.43393455],
        [1.42651684],
        [1.42164004],
        [1.41708667],
        [1.43587003],
        [1.43971513],
        [1.44844719],
        [1.44665906],
        [1.45384297],
        [1.45203935],
        [1.45080027],
        [1.44215346],
        [1.43597346],
        [1.44010899],
        [1.43032319],
        [1.426

In [22]:
# WORKING
lstm = Models.train_model(lstm_fold[0], 'lstm', config['regression_ensemble']['models'][1]['lstm'])

In [13]:
lstm

{'model': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fbab1b6c710>,
 'predictions': array([[1.3153099],
        [1.2913768],
        [1.2570856],
        [1.2254564],
        [1.2072307],
        [1.197623 ],
        [1.1961447],
        [1.2015517],
        [1.2086174],
        [1.2327015],
        [1.2495909],
        [1.2591982],
        [1.2726289],
        [1.2710193],
        [1.2626667],
        [1.2457758],
        [1.2239125],
        [1.2073878],
        [1.2004906],
        [1.2043546],
        [1.217895 ],
        [1.2266781],
        [1.230435 ],
        [1.2217182],
        [1.2158856],
        [1.2250061],
        [1.2291629],
        [1.2292991],
        [1.2259912],
        [1.2091273],
        [1.197242 ],
        [1.1903956],
        [1.1918502],
        [1.1999761],
        [1.2085423],
        [1.2309601],
        [1.253686 ],
        [1.2669461],
        [1.2729948],
        [1.2683965],
        [1.252851 ],
        [1.2483627],
        [1.241925 ]

In [14]:
# WORKING
tcn =Models.train_model(lstm_fold[1], 'tcn', config['regression_ensemble']['models'][2]['tcn'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(30, 4, 25)]             0         
_________________________________________________________________
residual_block_0 (ResidualBl [(30, 4, 64), (30, 4, 64) 13184     
_________________________________________________________________
residual_block_1 (ResidualBl [(30, 4, 64), (30, 4, 64) 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(30, 4, 64), (30, 4, 64) 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(30, 4, 64), (30, 4, 64) 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(30, 4, 64), (30, 4, 64) 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(30, 4, 64), (30, 4, 64) 16512 

In [15]:
tcn

{'model': <tensorflow.python.keras.engine.training.Model at 0x7fba88895cd0>,
 'predictions': array([[0.85743946],
        [0.8109147 ],
        [0.9330025 ],
        [0.79334223],
        [0.95745754],
        [0.7577297 ],
        [0.95525694],
        [0.810549  ],
        [0.8465721 ],
        [0.811553  ],
        [0.8166094 ],
        [0.8109147 ],
        [0.9231422 ],
        [0.90832883],
        [1.0478309 ],
        [0.9994899 ],
        [0.69491124],
        [0.93697166],
        [0.9171777 ],
        [0.93250865],
        [0.8300511 ],
        [0.94231147],
        [0.8896935 ],
        [0.91016835],
        [0.92764866],
        [0.83639926],
        [0.9092929 ],
        [1.0595646 ],
        [0.67388684],
        [1.0147051 ],
        [0.8282031 ],
        [0.83639926],
        [0.96068865],
        [0.9526321 ],
        [0.9994899 ],
        [0.8857041 ],
        [0.95745754],
        [1.0053073 ],
        [0.882899  ],
        [0.60099244],
        [0.917442  ],
      

# [Time Based Cross Validation](https://towardsdatascience.com/time-based-cross-validation-d259b13d42b8)
https://github.com/orhermansaffar/TimeBasedCV

In [16]:
import pandas as pd
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import *

class TimeBasedCV(object):
    '''
    Parameters 
    ----------
    train_period: int
        number of time units to include in each train set
        default is 30
    test_period: int
        number of time units to include in each test set
        default is 7
    freq: string
        frequency of input parameters. possible values are: days, months, years, weeks, hours, minutes, seconds
        possible values designed to be used by dateutil.relativedelta class
        deafault is days
    '''
    
    
    def __init__(self, train_period=30, test_period=7, freq='days'):
        self.train_period = train_period
        self.test_period = test_period
        self.freq = freq

        
        
    def split(self, data, validation_split_date=None, date_column='record_date', gap=0):
        '''
        Generate indices to split data into training and test set
        
        Parameters 
        ----------
        data: pandas DataFrame
            your data, contain one column for the record date 
        validation_split_date: datetime.date()
            first date to perform the splitting on.
            if not provided will set to be the minimum date in the data after the first training set
        date_column: string, deafult='record_date'
            date of each record
        gap: int, default=0
            for cases the test set does not come right after the train set,
            *gap* days are left between train and test sets
        
        Returns 
        -------
        train_index ,test_index: 
            list of tuples (train index, test index) similar to sklearn model selection
        '''
        
        # check that date_column exist in the data:
        try:
            data[date_column]
        except:
            raise KeyError(date_column)
                    
        train_indices_list = []
        test_indices_list = []

        if validation_split_date==None:
            validation_split_date = data[date_column].min().date() + eval('relativedelta('+self.freq+'=self.train_period)')
        
        start_train = validation_split_date - eval('relativedelta('+self.freq+'=self.train_period)')
        end_train = start_train + eval('relativedelta('+self.freq+'=self.train_period)')
        start_test = end_train + eval('relativedelta('+self.freq+'=gap)')
        end_test = start_test + eval('relativedelta('+self.freq+'=self.test_period)')

        while end_test < data[date_column].max().date():
            # train indices:
            cur_train_indices = list(data[(data[date_column].dt.date>=start_train) & 
                                     (data[date_column].dt.date<end_train)].index)

            # test indices:
            cur_test_indices = list(data[(data[date_column].dt.date>=start_test) &
                                    (data[date_column].dt.date<end_test)].index)
            
            #print("Train period:",start_train,"-" , end_train, ", Test period", start_test, "-", end_test,
            #      "# train records", len(cur_train_indices), ", # test records", len(cur_test_indices))

            train_indices_list.append(cur_train_indices)
            test_indices_list.append(cur_test_indices)

            # update dates:
            start_train = start_train + eval('relativedelta('+self.freq+'=self.test_period)')
            end_train = start_train + eval('relativedelta('+self.freq+'=self.train_period)')
            start_test = end_train + eval('relativedelta('+self.freq+'=gap)')
            end_test = start_test + eval('relativedelta('+self.freq+'=self.test_period)')

        # mimic sklearn output  
        index_output = [(train,test) for train,test in zip(train_indices_list,test_indices_list)]

        self.n_splits = len(index_output)
        
        return index_output
    
    
    def get_n_splits(self):
        """Returns the number of splitting iterations in the cross-validator
        Returns
        -------
        n_splits : int
            Returns the number of splitting iterations in the cross-validator.
        """
        return self.n_splits

In [17]:
dataset

,open,high,low,close,volume,label,stock,stocd,stocsd,momentum,...,rsi,obv,ma,bias6,psy12,asy5,asy4,asy3,asy2,asy1
Date_Timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-21,1.41115,1.41422,1.40247,1.40979,78980.100041,1.41361,13.189189,9.578919,19.849579,-0.02655,...,29.873956,-159980.600046,1.425180,-0.012336,41.7,-0.403064,-0.471831,-0.715772,-0.637503,-0.099256
2010-01-22,1.40977,1.41805,1.40837,1.41361,76660.300037,1.41517,20.072072,13.339429,14.445538,-0.02644,...,33.983332,-83320.300009,1.420572,-0.006774,41.7,-0.323346,-0.469180,-0.334804,0.085670,0.270596
2010-01-24,1.41557,1.41705,1.41396,1.41517,3128.600004,1.41458,22.882883,18.714715,13.877688,-0.01638,...,35.641946,-80191.700005,1.415528,-0.003171,33.3,-0.353285,-0.223529,0.093878,0.190445,0.110295
2010-01-25,1.41498,1.41939,1.41248,1.41458,79577.200048,1.40808,21.819820,21.591592,17.881912,-0.02625,...,35.280916,-159768.900053,1.412868,-0.000558,33.3,-0.187163,0.059984,0.113064,0.034298,-0.041700
2010-01-26,1.41454,1.41780,1.40397,1.40808,79232.600053,1.40280,10.108108,18.270270,19.525526,-0.03823,...,31.495786,-239001.500106,1.412246,-0.002826,25.0,-0.044125,-0.030342,-0.130655,-0.251130,-0.460559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-25,1.10873,1.10953,1.10730,1.10944,4887.310000,1.11012,23.116313,24.212506,24.694448,0.00301,...,50.839767,423499.922421,1.108610,0.000192,66.7,-0.051671,0.039465,0.042691,0.012170,0.064017
2019-12-26,1.10944,1.11088,1.10821,1.11012,52487.634115,1.11713,26.256564,24.209564,24.453872,0.00089,...,52.151783,475987.556536,1.109096,0.001135,75.0,0.043827,0.047337,0.028538,0.062645,0.061273
2019-12-27,1.11013,1.11883,1.10987,1.11713,124575.902401,1.11813,78.844711,42.739196,30.387089,0.00388,...,63.088379,600563.458936,1.110918,0.006029,75.0,0.163765,0.178773,0.251589,0.345376,0.629478


In [18]:
# How to use TimeBasedCV
data_for_modeling=dataset.reset_index()
tscv = TimeBasedCV(train_period=30,
                   test_period=7,
                   freq='days')
for train_index, test_index in tscv.split(data_for_modeling, date_column='Date_Timestamp'):
    continue

# get number of splits
tscv.get_n_splits()

514

In [19]:


#### Example- compute average test sets score: ####
X = data_for_modeling[['Close',columns]]
y = data_for_modeling[label]
from sklearn.linear_model import LinearRegression
import numpy as np

scores = []
for train_index, test_index in tscv.split(X, validation_split_date=datetime.date(2019,2,1)):

    data_train   = X.loc[train_index].drop('record_date', axis=1)
    target_train = y.loc[train_index]

    data_test    = X.loc[test_index].drop('record_date', axis=1)
    target_test  = y.loc[test_index]

    # if needed, do preprocessing here

    clf = LinearRegression()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    # accuracy for the current fold only    
    r2score = clf.score(data_test,target_test)

    scores.append(r2score)

# this is the average accuracy over all folds
average_r2score = np.mean(scores)

NameError: name 'columns' is not defined